<a href="https://colab.research.google.com/github/hwangtaemin/Credit-Card-AI/blob/main/xgb_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import LabelEncoder

     |████████████████████████████████| 296kB 8.2MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 174kB 26.5MB/s 
     |████████████████████████████████| 112kB 35.1MB/s 
     |████████████████████████████████| 143kB 22.0MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=0c0ec05f4f6213862188d79e163a3ddf9ab745838a82148a744b7e5e36ccd059
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/MyDrive/DACON/credit/train.csv')
train = train.drop(['index'], axis=1)

test = pd.read_csv('/content/drive/MyDrive/DACON/credit/test.csv')
test = test.drop(['index'], axis=1)

submit = pd.read_csv('/content/drive/MyDrive/DACON/credit/sample_submission.csv')

## Data 전처리
### occyp_type

In [4]:
def occyptype_pre(data):
  data.loc[data["DAYS_EMPLOYED"] >= 0,'occyp_type'] = "No job"
  data["occyp_type"] = data["occyp_type"].fillna("unknown")
  return

In [5]:
occyptype_pre(train)
occyptype_pre(test)

### 빈도수 적은 data 처리

In [6]:
# 빈도수가 적은 카테고리 확인(1% 기준)
# staff 직군으로 묶기(빈도수가 낮은 집단이 모두 staff -> sales staff로 묶어서 진행해보자)
def find_minority_cats(cat_cols, data_df, composite_category='Sales staff', threshold=0.03):
    """ Find minority categories for each feature column, and create a 
        dictionary that maps those to selected composite category """
    minority_col_dict = {}
    minority_mapping_dict = {}
    
    # find all feature categories with less than 1% proportion
    for feature in cat_cols:
        minority_col_dict[feature] = []
        minority_mapping_dict[feature] = {}
        
        for category, proportion in data_df[feature].value_counts(normalize=True).iteritems():
            if proportion < threshold:
                minority_col_dict[feature].append(category)
                
                # map those minority cats to chosen composite feature
                minority_mapping_dict[feature] = { x : composite_category for x 
                                                  in minority_col_dict[feature]}
                
    return minority_mapping_dict, minority_col_dict

In [7]:
cat_min_mappings, minority_cols = find_minority_cats(["occyp_type"], train)
minority_cols

{'occyp_type': ['Cooking staff',
  'Security staff',
  'Cleaning staff',
  'Private service staff',
  'Low-skill Laborers',
  'Waiters/barmen staff',
  'Secretaries',
  'Realty agents',
  'HR staff',
  'IT staff']}

In [8]:
cat_min_mappings

{'occyp_type': {'Cleaning staff': 'Sales staff',
  'Cooking staff': 'Sales staff',
  'HR staff': 'Sales staff',
  'IT staff': 'Sales staff',
  'Low-skill Laborers': 'Sales staff',
  'Private service staff': 'Sales staff',
  'Realty agents': 'Sales staff',
  'Secretaries': 'Sales staff',
  'Security staff': 'Sales staff',
  'Waiters/barmen staff': 'Sales staff'}}

In [9]:
train["occyp_type"] = train["occyp_type"].replace(cat_min_mappings["occyp_type"])
test["occyp_type"] = test["occyp_type"].replace(cat_min_mappings["occyp_type"])

In [10]:
special_type = ['High skill tech staff','Medicine staff','Accountants']

def specialist(data):
  data.loc[data["occyp_type"] == 'High skill tech staff','occyp_type'] = "specialist"
  data.loc[data["occyp_type"] == 'Medicine staff','occyp_type'] = "specialist"
  data.loc[data["occyp_type"] == 'Accountants','occyp_type'] = "specialist"

  data["occyp_type"] = data["occyp_type"].fillna("unknown")
  return

In [11]:
specialist(train)

In [12]:
specialist(test)

In [13]:
train["occyp_type"].unique()

array(['unknown', 'Laborers', 'Managers', 'Sales staff', 'specialist',
       'Core staff', 'Drivers', 'No job'], dtype=object)

## Data 전처리
### Feature selection

In [14]:
train= train.drop(['child_num', 'FLAG_MOBIL', 'house_type', 'edu_type', 'gender', 'phone', 'email', 'family_type', 'family_size', 'car', 'work_phone'],axis=1)
test= test.drop(['child_num', 'FLAG_MOBIL', 'house_type', 'edu_type', 'gender', 'phone', 'email', 'family_type', 'family_size', 'car', 'work_phone'],axis=1)

In [15]:
train.count()

reality          26457
income_total     26457
income_type      26457
DAYS_BIRTH       26457
DAYS_EMPLOYED    26457
occyp_type       26457
begin_month      26457
credit           26457
dtype: int64

In [17]:
#Binary variables
def binary(data):
  print('Having house reality or not: ')
  data['reality'] = data['reality'].replace(['N','Y'],[0,1])
  print(data['reality'].value_counts())
  print('--------------')


In [18]:
binary(train)

Having house reality or not: 
1    17830
0     8627
Name: reality, dtype: int64
--------------


In [19]:
binary(test)

Having house reality or not: 
1    6676
0    3324
Name: reality, dtype: int64
--------------


In [20]:
def preprocess(data):
  label_encoder = LabelEncoder()
  data['income_type']=label_encoder.fit_transform(data['income_type'])
  data['occyp_type']=label_encoder.fit_transform(data['occyp_type'])
  return

In [21]:
preprocess(train)
preprocess(test)

In [22]:
train

,reality,income_total,income_type,DAYS_BIRTH,DAYS_EMPLOYED,occyp_type,begin_month,credit
0,0,202500.0,0,-13899,-4709,7,-6.0,1.0
1,1,247500.0,0,-11380,-1540,2,-5.0,1.0
2,1,450000.0,4,-19087,-4434,3,-22.0,2.0
3,1,202500.0,0,-15088,-2092,5,-37.0,0.0
4,1,157500.0,2,-15037,-2105,3,-26.0,2.0
...,...,...,...,...,...,...,...,...
26452,0,225000.0,2,-12079,-1984,0,-2.0,1.0
26453,1,180000.0,4,-15291,-2475,7,-47.0,2.0
26454,0,292500.0,4,-10082,-2015,0,-25.0,2.0
26455,1,171000.0,4,-10145,-107,2,-59.0,2.0


### Normalization

In [23]:
continuos_variable = ['income_total','DAYS_BIRTH', 'DAYS_EMPLOYED','begin_month']
train_continuos = train[continuos_variable]
test_continuos = test[continuos_variable]
train_continuos_copy = train[continuos_variable]
test_continuos_copy = test[continuos_variable]

In [24]:
train_continuos_copy.columns

Index(['income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'begin_month'], dtype='object')

In [25]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train_continuos)
train_continuos_scaled = scaler.transform(train_continuos)
test_continuos_scaled = scaler.transform(test_continuos)

train_df_continuos_scaled = pd.DataFrame(train_continuos_scaled, columns=train_continuos_copy.columns, index=list(train_continuos_copy.index.values))
test_df_continuos_scaled = pd.DataFrame(test_continuos_scaled, columns=test_continuos_copy.columns, index=list(test_continuos_copy.index.values))


In [26]:
train_df_continuos_scaled

,income_total,DAYS_BIRTH,DAYS_EMPLOYED,begin_month
0,0.149136,0.490075,-0.463930,1.215231
1,0.590848,1.089621,-0.440878,1.275620
2,2.578550,-0.744719,-0.461929,0.249003
3,0.149136,0.207081,-0.444893,-0.656836
4,-0.292575,0.219220,-0.444988,0.007446
...,...,...,...,...
26452,0.369992,0.923252,-0.444108,1.456788
26453,-0.071719,0.158765,-0.447679,-1.260729
26454,1.032559,1.398558,-0.444333,0.067835
26455,-0.160062,1.383563,-0.430454,-1.985400


In [27]:
train = train.drop(columns=continuos_variable)
test = test.drop(columns=continuos_variable)
train = pd.concat([train,train_df_continuos_scaled], axis = 1)
test = pd.concat([test,test_df_continuos_scaled], axis = 1)

In [28]:
train

,reality,income_type,occyp_type,credit,income_total,DAYS_BIRTH,DAYS_EMPLOYED,begin_month
0,0,0,7,1.0,0.149136,0.490075,-0.463930,1.215231
1,1,0,2,1.0,0.590848,1.089621,-0.440878,1.275620
2,1,4,3,2.0,2.578550,-0.744719,-0.461929,0.249003
3,1,0,5,0.0,0.149136,0.207081,-0.444893,-0.656836
4,1,2,3,2.0,-0.292575,0.219220,-0.444988,0.007446
...,...,...,...,...,...,...,...,...
26452,0,2,0,1.0,0.369992,0.923252,-0.444108,1.456788
26453,1,4,7,2.0,-0.071719,0.158765,-0.447679,-1.260729
26454,0,4,0,2.0,1.032559,1.398558,-0.444333,0.067835
26455,1,4,2,2.0,-0.160062,1.383563,-0.430454,-1.985400


In [29]:
test

,reality,income_type,occyp_type,income_total,DAYS_BIRTH,DAYS_EMPLOYED,begin_month
0,0,1,4,-0.734287,-1.435662,2.227162,-2.045789
1,1,2,0,-0.513431,-0.715444,-0.492750,-0.596447
2,1,4,2,-1.157623,0.016912,-0.431254,-0.838004
3,0,0,1,-0.734287,-0.788275,-0.448087,-0.898393
4,1,2,3,0.369992,-0.443637,-0.497944,1.094452
...,...,...,...,...,...,...,...
9995,1,4,6,0.149136,-0.627143,-0.469204,0.430170
9996,1,4,2,0.149136,1.207198,-0.439241,-0.475668
9997,1,4,6,1.032559,-1.203840,-0.531645,-1.743843
9998,0,0,7,-0.071719,-0.138747,-0.437568,-0.415279


In [30]:
train = train.reindex(['reality',	'income_total',	'income_type',	'DAYS_BIRTH',	'DAYS_EMPLOYED',	'occyp_type',	'begin_month', 	'credit'], axis=1)
test = test.reindex(['reality',	'income_total',	'income_type',	'DAYS_BIRTH',	'DAYS_EMPLOYED',	'occyp_type',	'begin_month'], axis=1)

In [31]:
train

,reality,income_total,income_type,DAYS_BIRTH,DAYS_EMPLOYED,occyp_type,begin_month,credit
0,0,0.149136,0,0.490075,-0.463930,7,1.215231,1.0
1,1,0.590848,0,1.089621,-0.440878,2,1.275620,1.0
2,1,2.578550,4,-0.744719,-0.461929,3,0.249003,2.0
3,1,0.149136,0,0.207081,-0.444893,5,-0.656836,0.0
4,1,-0.292575,2,0.219220,-0.444988,3,0.007446,2.0
...,...,...,...,...,...,...,...,...
26452,0,0.369992,2,0.923252,-0.444108,0,1.456788,1.0
26453,1,-0.071719,4,0.158765,-0.447679,7,-1.260729,2.0
26454,0,1.032559,4,1.398558,-0.444333,0,0.067835,2.0
26455,1,-0.160062,4,1.383563,-0.430454,2,-1.985400,2.0


In [32]:
test

,reality,income_total,income_type,DAYS_BIRTH,DAYS_EMPLOYED,occyp_type,begin_month
0,0,-0.734287,1,-1.435662,2.227162,4,-2.045789
1,1,-0.513431,2,-0.715444,-0.492750,0,-0.596447
2,1,-1.157623,4,0.016912,-0.431254,2,-0.838004
3,0,-0.734287,0,-0.788275,-0.448087,1,-0.898393
4,1,0.369992,2,-0.443637,-0.497944,3,1.094452
...,...,...,...,...,...,...,...
9995,1,0.149136,4,-0.627143,-0.469204,6,0.430170
9996,1,0.149136,4,1.207198,-0.439241,2,-0.475668
9997,1,1.032559,4,-1.203840,-0.531645,6,-1.743843
9998,0,-0.071719,0,-0.138747,-0.437568,7,-0.415279


In [33]:
X=train.drop('credit',axis=1)
y=train['credit']

In [37]:
def objective(trial,train=train,target=y):

  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

  param={
      'max_depth':trial.suggest_int('max_depth',25,30),
      'learning_rate':0.03,
      'subsample':trial.suggest_categorical('subsample',[0.5,0.6,0.7,0.8,0.9]),
      'colsample_bytree':trial.suggest_categorical('colsample_bytree',[0.5,0.6,0.7,0.8,0.9]),
      'eval_metric': 'mlogloss',
      'objective' : 'multi:softprob',
      'random_state': 42,
      'n_estimators': 1000,
  }

  model=XGBClassifier(**param)

  model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=30,verbose=False)

  predictions=model.predict_proba(X_test)

  logloss = log_loss(y_test,predictions)
  return logloss

In [38]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
study.best_params

[I 2021-05-26 01:04:27,515] A new study created in memory with name: no-name-86c6f15e-417b-4999-8835-479bf38be100
[I 2021-05-26 01:04:52,828] Trial 0 finished with value: 0.7089193310428098 and parameters: {'max_depth': 26, 'subsample': 0.5, 'colsample_bytree': 0.5}. Best is trial 0 with value: 0.7089193310428098.
[I 2021-05-26 01:05:17,540] Trial 1 finished with value: 0.712405888484094 and parameters: {'max_depth': 25, 'subsample': 0.6, 'colsample_bytree': 0.6}. Best is trial 0 with value: 0.7089193310428098.
[I 2021-05-26 01:05:47,011] Trial 2 finished with value: 0.7239958666671893 and parameters: {'max_depth': 28, 'subsample': 0.5, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.7089193310428098.
[I 2021-05-26 01:06:15,315] Trial 3 finished with value: 0.7290743253998407 and parameters: {'max_depth': 29, 'subsample': 0.8, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.7089193310428098.
[I 2021-05-26 01:06:38,632] Trial 4 finished with value: 0.7114290886742323 and

{'colsample_bytree': 0.5, 'max_depth': 29, 'subsample': 0.7}

In [39]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))


In [40]:
import random
random.seed(42)
xgb_models={}
for fold in range(10):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    xgb = XGBClassifier(n_estimators=1000,
                        objective='multi:softprob',
                        eval_metric='mlogloss',
                        learning_rate=0.01,
                        max_depth=29,
                        subsample=0.7,
                        colsample_bytree=0.5,
                        )
    xgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=100,
           verbose=100)
    xgb_models[fold]=xgb
    print(f'================================================================================\n\n')

====================================1============================================
[0]	validation_0-mlogloss:1.09197	validation_1-mlogloss:1.0935
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[100]	validation_0-mlogloss:0.731487	validation_1-mlogloss:0.829173
[200]	validation_0-mlogloss:0.567986	validation_1-mlogloss:0.737972
[300]	validation_0-mlogloss:0.47292	validation_1-mlogloss:0.700781
[400]	validation_0-mlogloss:0.410983	validation_1-mlogloss:0.685794
[500]	validation_0-mlogloss:0.366199	validation_1-mlogloss:0.679643
[600]	validation_0-mlogloss:0.329404	validation_1-mlogloss:0.678095
Stopping. Best iteration:
[599]	validation_0-mlogloss:0.330079	validation_1-mlogloss:0.678051



====================================2============================================
[0]	validation_0-mlogloss:1.09216	validation_1-mlogloss:1.0936
Multiple eval metrics have been passed

In [41]:
test.rename(columns = {'reality' : 'f0','income_total' : 'f1', 'income_type' : 'f2', 'DAYS_BIRTH' : 'f3', 'DAYS_EMPLOYED' : 'f4', 'occyp_type' : 'f5', 'begin_month' : 'f6'}, inplace = True)

In [42]:
test

,f0,f1,f2,f3,f4,f5,f6
0,0,-0.734287,1,-1.435662,2.227162,4,-2.045789
1,1,-0.513431,2,-0.715444,-0.492750,0,-0.596447
2,1,-1.157623,4,0.016912,-0.431254,2,-0.838004
3,0,-0.734287,0,-0.788275,-0.448087,1,-0.898393
4,1,0.369992,2,-0.443637,-0.497944,3,1.094452
...,...,...,...,...,...,...,...
9995,1,0.149136,4,-0.627143,-0.469204,6,0.430170
9996,1,0.149136,4,1.207198,-0.439241,2,-0.475668
9997,1,1.032559,4,-1.203840,-0.531645,6,-1.743843
9998,0,-0.071719,0,-0.138747,-0.437568,7,-0.415279


In [43]:
submit.iloc[:,1:]=0
for fold in range(10):
    submit.iloc[:,1:] += xgb_models[fold].predict_proba(test)/10

In [44]:
submit.to_csv('xgb_final.csv', index=False)